In [4]:
# %%capture
# !pip install selenium

## 지금까지 나온 문제
    대문 img를 말고 하단에 img가 추가 되어있는 책과 아닌책이 있어서 한칸씩 밀리는 경우
    책 속으로 라고하는 챕터가 있거나 없거나
    XPATH를 사용하지 않고 get_attribute('class')로 찾아야할지?
    ------

    추천사,책속으로,출판사서평,혹은 없는경우도 있음. div[10]
    
    * 형태 예제
        책소개 - 작가정보 - 목차 - 출판사서평
        책소개 - 작가정보(번역자까지 2개) - 목차 - 추천사 - 출판사서평
        책소개 - 작가정보 - 목차 - 책 속으로
        책소개 - 작가정보 - 목차 - 책 속으로 - 출판사 서평
        

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import re
from bs4 import BeautifulSoup as Bs
import requests
import lxml.html

### request / bs

In [12]:
url = 'https://product.kyobobook.co.kr/detail/S000200926912'
r = requests.get(url)
r.status_code
bs = Bs(r.content,'html.parser')
bs


<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

<meta charset="utf-8"/>
<html xmlns="http://www.w3.org/1999/xhtml">
<head>
<title> 서비스 점검 - 교보문고 </title>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, minimum-scale=1.0, user-scalable=no, target-densitydpi=medium-dpi" name="viewport"/>
<meta content="" name="author">
<meta content="" name="keywords">
<meta content="" name="description"/>
<style type="text/css">
	* {padding:0;margin:0;list-style:none;}
  </style>
</meta></meta></head>
<body>
<style type="text/css">
	.errorPage_wrap {width:680px;margin:117px auto 0;}
	.errorPage_wrap .logo {font-size: 0; line-height: 0; text-align:center;}
	.errorPage_wrap .title {font-size:45px; color:#333; margin:23px 0 0 0; text-align:center;}
	.errorPage_wrap .title strong{color:#ff6600;}

	.errorPage_wrap .errorMsg01_wrap {margin:45px 0 46px;text-align:center; font-size:18px; line-height:1.4; color:#

### selenium html source code

In [44]:
driver = webdriver.Chrome()
# driver.get('https://product.kyobobook.co.kr/new/#?page=1&sort=new&year=2023&month=04&week=3&per=20&saleCmdtDvsnCode=KOR&gubun=newGubun&saleCmdtClstCode=')
# driver.get('https://product.kyobobook.co.kr/detail/S000201423559')
# driver.get('https://product.kyobobook.co.kr/detail/S000201470540') # 아웃풋 법칙
# driver.get('https://product.kyobobook.co.kr/detail/S000201478042')
driver.get('https://product.kyobobook.co.kr/detail/S000200926912')

In [18]:
html = '''
<html><div><img src="https://jigoobooks.smilecast.co.kr/Book_detail/9791191891256_detail.jpg" onerror="this.style.display='none';"/><br></div><div class="product_detail_area book_contents"><div class="title_wrap title_size_md"><h2 class="title_heading">목차</h2></div><div class="auto_overflow_wrap overflow"><div class="auto_overflow_contents"><div class="auto_overflow_inner"><ul class="book_contents_list"><li class="book_contents_item">프롤로그<br>제1장 나는 나를 사랑해<br>오둥이입니다만 / 예상치 못한 행복 / 빗방울 / 무지개 / 오두막 / 책 / 만남 / 동행<br><br>제2장 네가 곁에 있어서 세상은 빛나<br>초상화 / 요거트 / 식빵 / 가만히 있어도 좋아 / 벚꽃놀이 / 풍선껌 / 모자 / 숟가락 / 더위 / 다이빙 / 우리는 늘 서로의 곁에 있어 / 신발 / 기다리면 지나갈 거야 / 정상 / 탄산음료 / 날다 / 간지럼<br><br>제3장 모든 순간에 행복이 있어<br>나, 너 그리고 우리 / 의사 / 목걸이 / 접착제 / 반쪽 / 끼릭끼릭 / 귤 / 뿡 / 인형 / 진심을 전하다 / 군고구마 / 유성매직 / 항상 고마워 / 꿀 / 코골이 / 눈더미 / 내가 초라해 보일 때 / 선물<br><br>에필로그<br>부록</li></ul></div></div><div class="auto_overflow_footer"><button type="button" class="btn_more_body" data-btn-toggle=""><span class="text"></span><span class="ico_arw"></span></button></div></div></div><div class="product_detail_area book_publish_review"><div class="title_wrap title_size_md"><h2 class="title_heading">출판사 서평</h2></div><div class="auto_overflow_wrap type_text overflow"><div class="auto_overflow_contents"><div class="auto_overflow_inner"><p class="info_text">오늘을 사는 오리 오둥이의 무한한 귀여움과<br>용기, 응원을 꾹꾹 눌러담은 최고의 긍정툰!<br><br>당차게 계획한 새해 목표도 팍팍한 현실에 금세 사라져버리는 요즘. 숱한 응원의 메시지도, 격려도 때로는 버겁기만 한 이들에게 오둥이는 현재의 소중함을 말한다. 험난한 여정 앞에서도 오뚝이처럼 일어서는 의지, 댕청한 눈빛으로 상황을 금방 받아들이며 나아가는 장면들은 우리 앞에 닥친 크고 작은 불운들도 어쩌면 그저 물 흐르듯 받아들이고 흘려보내야 함을 일깨워주는 듯하다.<br>하루하루 치열하고 고단하게 사는 삶 속에서도 우리가 스스로를 잃지 않으려면, 오둥이의 자세가 필요하지 않을까? 나를 사랑하고, 지금, 여기, 이곳에서 일어나는 일들을 바라보고 그 과정에서 찾게 된 한 명의 친구를 진정으로 아끼며 사랑하는 마음. 비를 맞고, 넘어지고, 길을 잃는 과정 안에서도 계속 나아갈 수 있는 건, 이런 마음들이 오둥이의 둥그런 몸을 지탱해주고 있기 때문일 것이다.<br>오둥이의 무한한 귀여움과 용기, 응원을 꾹꾹 눌러 담아 긍정의 기운을 보내는 《오둥이입니다만!》. 부담스러운 격려보다 가볍지만 묵직하게 마음을 채울 따뜻함이 필요하다면, 오둥이 에세이를 읽고 제대로 힐링하길 바란다.</p></div></div><div class="auto_overflow_footer"><button type="button" class="btn_more_body" data-btn-toggle=""><span class="text"></span><span class="ico_arw"></span></button></div></div></div></html>
'''

Bs(html,'html.parser')


<html><div><img onerror="this.style.display='none';" src="https://jigoobooks.smilecast.co.kr/Book_detail/9791191891256_detail.jpg"/><br/></div><div class="product_detail_area book_contents"><div class="title_wrap title_size_md"><h2 class="title_heading">목차</h2></div><div class="auto_overflow_wrap overflow"><div class="auto_overflow_contents"><div class="auto_overflow_inner"><ul class="book_contents_list"><li class="book_contents_item">프롤로그<br/>제1장 나는 나를 사랑해<br/>오둥이입니다만 / 예상치 못한 행복 / 빗방울 / 무지개 / 오두막 / 책 / 만남 / 동행<br/><br/>제2장 네가 곁에 있어서 세상은 빛나<br/>초상화 / 요거트 / 식빵 / 가만히 있어도 좋아 / 벚꽃놀이 / 풍선껌 / 모자 / 숟가락 / 더위 / 다이빙 / 우리는 늘 서로의 곁에 있어 / 신발 / 기다리면 지나갈 거야 / 정상 / 탄산음료 / 날다 / 간지럼<br/><br/>제3장 모든 순간에 행복이 있어<br/>나, 너 그리고 우리 / 의사 / 목걸이 / 접착제 / 반쪽 / 끼릭끼릭 / 귤 / 뿡 / 인형 / 진심을 전하다 / 군고구마 / 유성매직 / 항상 고마워 / 꿀 / 코골이 / 눈더미 / 내가 초라해 보일 때 / 선물<br/><br/>에필로그<br/>부록</li></ul></div></div><div class="auto_overflow_footer"><button class="btn_more_body" data-btn-toggle="" type="button"><span class="text"></span><span 

In [10]:
sel_html = driver.page_source
bs = Bs(sel_html,'html.parser')
bs

<html class="kbb_loaded" data-service="product" data-view="ink" lang="ko"><head><script async="" src="https://www.google-analytics.com/gtm/optimize-dyn.js?id=OPT-TXV7NFN&amp;cid=1529319710.1684854360&amp;cb=0.4487810692049221" type="text/javascript"></script><script async="" crossorigin="anonymous" src="https://connect.facebook.net/ko_KR/sdk.js?hash=4a856a1673e7e37ea04360e022951cbe"></script><script async="" src="https://www.googletagmanager.com/gtag/js?id=G-CQHKV7VZV7&amp;l=dataLayer&amp;cx=c" type="text/javascript"></script><script async="" id="www-widgetapi-script" src="https://www.youtube.com/s/player/e50626d8/www-widgetapi.vflset/www-widgetapi.js" type="text/javascript"></script><script async="" src="https://www.googletagmanager.com/gtag/destination?id=G-CQHKV7VZV7&amp;l=dataLayer&amp;cx=c" type="text/javascript"></script><script async="" src="https://www.clarity.ms/s/0.7.8/clarity.js"></script><script async="" gtm="GTM-TWGDFSP" src="https://www.googleoptimize.com/optimize.js?id=O

In [4]:
html = driver.find_element(By.XPATH,'//*[@id="scrollSpyProdInfo"]/div[4]')
html.get_attribute('img')

### selenium

In [54]:
for i in driver.find_elements(By.XPATH,'//*[@id="scrollSpyProdInfo"]/div'):
    print(i.text)


북카드
함께 구매한 책
나의 돈 많은 고등학교 친구
송희구
16,200원
엣지 한 끗의 차이를 만드는 내 안의 힘
로라 후앙. 이윤진
16,020원
숨 쉴 때마다 새로운 내가 된다면
마셜 골드스미스. 마크 라이터. 안솔비
17,100원
디깅(Digging)
박치은
16,200원
신뢰의 끈을 놓치지 말라
헤롤드 셔먼. 김경희
15,750원
일머리 문해력
송숙희
15,120원
키워드 Pick

기브
리원
정체
저항
무력
단계
더보기

책 소개
이 책이 속한 분야
국내도서 > 자기계발 > 성공/처세 > 자기관리/처세
구독자 32만 명의 유튜브 〈푸릉_렘군〉 대표, 대한민국 최고의 부동산 전문가,
10만 베스트셀러 작가, 3040 직장인들의 멘토,
렘군의 첫 자기계발서 출간!
어느덧 직장경력 10여 년, 항상 최선을 다해 열심히 살아온 것 같은데 생각보다 손에 쥐고 있는 것이 별로 없다고 느끼는 사람들이 꽤 많다. 자기계발에 소홀한 것도 아니고, 책도 꽤 읽고 공부도 한 것 같은데 왜 이렇게 회사를 벗어나서는 할 줄 아는 게 없다고 느껴지는 걸까? 지난 5년간 렘군은 ‘시작 컨설팅’이라는 이름으로 평범한 사람들이 자신의 강점을 찾고 제2의 인생 설계를 할 수 있도록 도왔다. 그는 이 과정을 통해 얻은 지식을 ‘아웃풋(output)’이라는 단어로 설명한다. 아웃풋은 단순히 어떤 생산물 또는 결과물만을 의미하지 않는다. 저자는 이를 타인을 위해, 세상과 연결되기 위해 제공하는 생산물로 정리한다.
우리는 늘 자신이 완벽하지 않다고 생각해 강의를 듣고, 책을 읽으며 인풋만 일삼는다. 남들이 만들어 놓은 지식, 콘텐츠, 물건 등을 소비하는 데 급급해 자신이 무엇을 세상에 제공할 수 있는지 잊고 산다. 그러나 조금 부족하더라도, 완벽하지 않더라도 자신의 아웃풋을 세상에 내놓는 바로 그 순간, ‘소비자의 영역’에서 벗어나 ‘생산자의 영역’으로 진입하게 된다. 이때 비로소 삶에 변화가 일어나는 것이다. 이렇게 저자는 ‘아웃풋 법칙’을 6단계로 정리해 사람들이 쉽고 간단하게 생산자

In [68]:
#책소개 
#위에 이미지가 있을경우 한칸씩 밀려서 작성된다?
print(driver.find_element(By.XPATH,'//*[@id="scrollSpyProdInfo"]/div[4]').get_attribute('class'))

product_detail_area detail_img


In [55]:
def img_finder():
    #product_detail_area detail_img / product_detail_area book_intro
    standard = driver.find_element(By.XPATH,'//*[@id="scrollSpyProdInfo"]/div[4]').get_attribute('class')
    if standard == 'product_detail_area detail_img':
        pass
    elif standard == 'product_detail_area book_intro':
        pass
    else:
        return print('예외처리가 안된 XPATH')
        


In [5]:
# title = driver.find_element(By.XPATH,'//*[@id="contents"]/div[1]/div/div[1]/div/div[1]/div[1]/div/h1/span').text
title = driver.find_element(By.CLASS_NAME,'prod_title')
title.get_attribute('outerHTML')

'<span class="prod_title">아웃풋 법칙</span>'

In [28]:
o = driver.find_element(By.XPATH,'//*[@id="scrollSpyProdInfo"]/div[10]/div[1]').get_attribute('outerHTML')
o

'<div class="title_wrap title_size_md">\n        <h2 class="title_heading">목차</h2>\n    </div>'

In [30]:
outer = driver.find_element(By.XPATH,'//*[@id="scrollSpyProdInfo"]/div[10]/div[2]/div[1]').get_attribute('outerHTML')
outer


'<div class="auto_overflow_contents">\n            <div class="auto_overflow_inner">\n                <ul class="book_contents_list">\n                    <li class="book_contents_item">들어가는 글 <br><br>1부 거대 스태그플레이션과 부채 위기<br><br>1장 눈먼 시장이 불러들인 부채 위기 <br>벼랑 끝에 몰린 낙관주의자들 │ 반복되는 위기의 역사 │ 부채 팬데믹은 이미 시작되었다 <br><br>2장 민간 및 공공 부문 정책의 실패 <br>잘못된 정책이 낳은 세 가지 불일치 │ 부채 위기를 완전히 벗어날 방법은 없다 │ 부채의 빙산 아래 숨은 비용들 <br><br>3장 인구통계학적 시한폭탄 <br>세계의 고령화 그리고 바닥난 연금 │ 고령화 시대의 딜레마 │ 이민자 앞에 닫힌 문 <br><br>4장 저금리의 함정 그리고 호황과 불황의 주기 <br>호황과 불황의 주기는 게임이 아니다 │ 야성적 충동을 조장하는 느슨한 통화 정책 │ 금융 붕괴는 경제적 ‘실패’가 아닌 ‘인재’다 │ 좋은 디플레이션, 나쁜 디플레이션, 끔찍한 디플레이션 │ 쉬운 돈과 느슨한 정책의 함정에 빠진 중앙은행들<br><br>5장 거대 스태그플레이션의 도래 <br>1970년대 스태그플레이션의 악몽 │ 눈앞에 다가온 스태그플레이션의 징조들 │ 세계를 불황에 빠트릴 11가지 충격 <br><br>2부 금융, 무역, 지정학, 첨단기술, 환경의 위기<br><br>6장 통화 붕괴와 금융 불안 <br>달러의 무기화, 중국의 탈동조화 │ 암호화폐 그리고 탈중앙화 금융에 대한 그릇된 맹신 │ ‘가격’을 아는 것과 ‘가치’를 아는 것은 다르다 <br><br>7장 세계화의 종말 <br>일자리를 지키기 위한 탈세계화의 역효과 │ 세계 경제의 파이를 키운 자유무역 │ 사라지는 일자리에 분노하는 사람들 │ 글로벌 무역은 제로섬 게임이 아니다 <br><br>8장 

In [70]:
# sub_title = driver.find_element(By.XPATH,'//*[@id="contents"]/div[1]/div/div[1]/div/div[3]/div/div[1]/div/span').text
sub_title = driver.find_element(By.CLASS_NAME,'prod_desc').text
sub_title

'설계되지 않은 성공, K컬처산업의 운명을 바꾼 9가지 결정적 장면'

In [71]:
# author = driver.find_element(By.XPATH,'//*[@id="contents"]/div[1]/div/div[2]/div/div[1]/div/div[2]/div[1]/div/div/a').text
author = driver.find_element(By.CLASS_NAME,'author').text
# author = driver.find_element(By.XPATH,'//*[@id="contents"]/div[1]/div/div[2]/div/div[1]/div/div[1]/div[1]/div/div/a').text
# //*[@id="contents"]/div[1]/div/div[2]/div/div[1]/div/div[1]/div[1]/div/div/a img 있는 url
#//*[@id="contents"]/div[1]/div/div[2]/div/div[1]/div/div[2]/div[1]/div/div/a img 없는 url
author

'김윤지 저자(글)'

In [72]:
# price = driver.find_element(By.XPATH,'//*[@id="contents"]/div[1]/div/div[2]/div/div[3]/div[1]/div[2]/div').text
price = driver.find_element(By.CLASS_NAME,'prod_price_box').text
price

'10% 16,920원 18,800원'

In [73]:
img = driver.find_element(By.XPATH,'//*[@id="contents"]/div[1]/div/div[2]/div/div[2]/div[1]/div[1]/ul/li[1]/div/div[2]/img').get_attribute('src')
# img = driver.find_element(By.CLASS_NAME,'portrait_img_box portrait').get_attribute('src')
img

'https://contents.kyobobook.co.kr/sih/fit-in/458x0/pdt/9791167740977.jpg'

In [74]:
# cls = driver.find_element(By.XPATH,'//*[@id="scrollSpyProdInfo"]/div[4]/div[1]/ul/li/a[2]').text
cls = driver.find_elements(By.CLASS_NAME,'intro_category_link')[1].text
# cls = driver.find_element(By.XPATH,'//*[@id="scrollSpyProdInfo"]/div[5]/div[1]/ul/li/a[2]').text
# //*[@id="scrollSpyProdInfo"]/div[5]/div[1]/ul/li/a[2] img 있는 url
# //*[@id="scrollSpyProdInfo"]/div[4]/div[1]/ul/li/a[2] 없는 url
cls

'정치/사회'

In [75]:
# intro = driver.find_element(By.XPATH,'//*[@id="scrollSpyProdInfo"]/div[4]/div[2]/div').text
# intro = driver.find_element(By.XPATH,'//*[@id="scrollSpyProdInfo"]/div[5]/div[2]').text
intro = driver.find_element(By.CLASS_NAME,'intro_bottom').text
intro

'“우리가 잘한 걸까, 세계가 이상한 걸까?”\n세계를 놀라게 한 K컬처산업, 그 도약의 순간들에 관하여\n문화산업 연구자 김윤지가 정리한 30년 한류 막전막후\nBTS 멤버 지민의 빌보드 1위, 〈오징어 게임〉에서 〈더 글로리〉로 이어지는 넷플릭스 글로벌 1위 행진, 〈기생충〉과 윤여정의 오스카 수상을 보면 자연스럽게 이런 질문을 던지게 된다. “도대체 그동안 무슨 일이 일어난 걸까?” 거대한 외국자본과의 경쟁에서 살아남기 위해 발버둥 치던 변방의 문화산업이 세계를 강타하리란 걸 누가 알았을까? 할리우드 영화사들이 한국 영화시장에 직접 배급하는 걸 막기 위해 극장에 뱀을 풀던 1988년에 한국영화가 오스카를 휩쓰는 미래를 상상한 사람은 아무도 없었고, 지역 행사를 목표로 댄스음악을 만들던 1990년대에 빌보드 차트 진입은 꿈도 꿀 수 없는 일이었다. 그러나 상상보다 더 상상 같은 일들은 우리의 당연한 일상이 되었다. 친숙하지만 이해할 수 없는 이 현상을 이제는 명쾌하게 설명할 수 있어야 하지 않을까? 《한류 외전》은 영화시장 개방에서 BTS에 이르는 30여 년 동안 한국의 문화산업에 생긴 일들을 9개의 장면을 통해 설명한다.\n\n우리가 알지 못했던 K컬처산업의 역동적인 역사\n문화산업이 성공하기 위한 토양은 어떻게 만들어지는가?\n\nIMF 외환 위기가 없었다면 이를 극복하기 위한 새로운 산업 동력도 절실하지 않았을 것이고, 문화산업정책을 전폭적으로 입안하지 못했을 가능성이 높았다. 1980년대부터 이어져온 민주화 열기로 자유로운 창작 분위기를 조성하지 못했다면 우리 드라마, 영화의 수준도 계속 군부 시대의 틀 안에 놓였을 수도 있다. -‘프롤로그’ 중에서\n\n저자가 정리한 9개의 장면에는 한류와 연결하기 힘들었던 의외의 순간들이 존재한다. 아시아를 강타한 외환 위기와 한류드라마 사이에는 어떤 관계가 있을까? IMF 이후의 벤처캐피털은 박찬욱과 봉준호에게 어떤 기회를 주었을까? 위기에 몰린 가요 기획사는 왜 해외 진출을 선택할 수밖에 없었을까? 기회가 위

In [76]:
# contents = driver.find_element(By.XPATH,'//*[@id="scrollSpyProdInfo"]/div[8]/div[2]/div[1]/div/ul/li').text
# contents = driver.find_element(By.XPATH,'//*[@id="scrollSpyProdInfo"]/div[9]/div[2]/div[1]/div/ul/li').text
contents = driver.find_element(By.CLASS_NAME,'book_contents_item').text
contents


'프롤로그: K컬처산업의 토양은 무엇이었나?\n\n1장 시장개방이 만든 위기와 기회\n장면1 할리우드에 뱀으로 맞선 사람들\n세계시장의 빗장을 연 우루과이라운드 / 아무도 예상하지 못한 미래\n\n2장 〈쥬라기 공원〉에서 문화 융성까지, 새로운 산업 동력 찾기\n장면2 자동차 150만 대보다 강력한 공룡의 습격\n놀고먹는 일의 충격적인 경제효과 / 한류, 정부정책의 산물? 설계되지 않은 성공? / 불안한 세계화 시대, 딴따라에게서 미래를 찾다 / 팔길이 원칙, 지원은 하되 간섭하지 않는다 / 성장과 보호의 딜레마 / 국가의 도구가 된 한류\n\n3장 자유로운 드라마 산업이 만든 글로벌 공동체\n장면3 저렴하고 질 좋은 한국 드라마를 팝니다\n90년대 드라마의 질적 전환을 가져온 두 사건 / 대만이 한국 드라마 해외 진출의 최초 거점이 된 이유 / 드라마 산업화의 가능성을 제시한 〈겨울연가〉 / 아시아 시장을 넘어설 가능성을 보여준 〈대장금〉 / 초고속인터넷망의 비판적 시청자들 / 언어의 장벽을 뛰어넘는 한류 팬들 / 민주화, 자율성, 공정한 경쟁\n\n4장 낭만의 시대에서 투자의 시대로, 벤처 투자 대상이 된 영화\n장면4 지방 배급업자 김 사장이 한숨을 쉰 이유\n자본조달을 위한 유일한 선택지 / 삼성, LG, 대우, 영화시장에 진격한 대기업들 / IMF로 인한 1차 퇴각 / 새로운 영화 세대가 등장하다 / 문화산업 투자를 위해 총대를 멘 정부 / 영화로 눈을 돌린 투자자들 / 투자 촉진의 마중물이 된 영상전문투자조합 / 더 이상의 주먹구구는 없다, 금융계와 영화계의 신뢰 구축 / 한국영화 르네상스의 씨앗들\n\n5장 K팝 제조 시스템의 역동적인 시장 개척\n장면5 베이징 공인체육관을 불태운 H.O.T.\n신조어 ‘한류’는 어디에서 왔을까? / 빌보드 이전에 길보드가 있었다 / 10대 소비자를 위한 아이돌 제작 시스템 / 유통기간 7년의 아이돌 / MP3가 불러온 악몽 / 세계화 외에는 방법이 없다 / 난공불락의 미국 시장 / ‘미국식’으로 자리 잡고 싶었던

In [77]:
# inner_book = driver.find_element(By.XPATH,'//*[@id="scrollSpyProdInfo"]/div[10]/div[2]/div[1]/div/p').text
inner_book = driver.find_element(By.CLASS_NAME,'product_detail_area.book_inside').text
inner_book

'책 속으로\n전 세계가 거대한 하나의 시장으로 변화하는 치열하던 그 순간, 당황한 한국 영화인들은 영화관에 뱀을 풀었다. 콘크리트 바닥에 풀어놓은 뱀들이 힘을 쓰지 못하고 맥없이 죽는 모습이 말해 주듯 그게 정답이 아님은 뱀을 풀었던 영화인들도, 지켜보는 우리도 모두 알았다. 그런데 그런 강한 저항의 움직임은 정부도, 업계도 변화가 필요하다는 것을 감지하게 했다.(1장 시장개방이 만든 위기와 기회) -28~29쪽\n\n아마도 대기업이 든든히 시장에 버티고 있었다면 초기 시장 진출 과정은 조금 달랐을지도 모른다. 하지만 대기업이 없었기에 한국 드라마 산업은 누구나 성공할 수도, 누구나 실패할 수도 있는 공정한 경쟁의 장이 되었다. 대기업의 부재 덕분에 운동장은 평평했다. 향후 한류드라마 수준이 빠르게 향상된 데는 평평한 운동장에서 동등하게 경쟁하면서 키운 힘의 영향도 있었다.(3장 자유로운 드라마 산업이 만든 글로벌 공동체) -92쪽\n\nSM이 K팝 기획사로서 처음 코스닥에 입성한 것은 K팝 기획사가 정상적 수익모델을 가진 기업으로 인정받았다는 점에서 의의를 가진다. SM은 아이돌 육성과 해외 사업 확장을 위해 자본이 필요하다는 계획을 밝혔고, 시장이 이를 정상적 기업활동으로 인정했다는 의미기 때문이다. 즉 K팝 산업이 성장, 발전하기 위해 어떤 자본이 필요하고, 그런 활동을 하는 기업의 원형은 어떤 형태여야 하는지가 검증이 된 셈이다.(6장 엔터테인먼트도 안정적 산업이 될 수 있다) -193쪽\n\n음악적 진정성은 견고한 북미 음악시장의 장벽을 깨뜨리는 가장 중요한 비기였을까? 즉 과거의 K팝 아이돌도 음악적 진정성만 갖추었더라면 북미 시장의 벽을 뚫고 더 빨리 진입할 수 있었을지가 문제다. 꼭 그렇지는 않았을 것으로 보인다. K팝이 변화한 것만큼 미국 시장도 변화해 왔고, 그런 변화가 있었기에 BTS같은 아시아 아이돌에게도 기회가 생겼기 때문이다.(7장 팬덤이라는 세계화 전진 기지) -214쪽\n\n일본 대중문화 개방 방침이 준비되기 시작할 무렵 김대중 

In [79]:
recommend = driver.find_element(By.CLASS_NAME,'product_detail_area.book_recommend').text
recommend

'추천사\n정길화 (한국국제문화교류진흥원장, 전 MBC PD•중남미지사장)\n실증적인 자료와 검증, 때로는 감동적인 서사를 통해 한국 문화산업의 역동성을 구성한다. 《한류 외전》의 출현은 이제 본격적인 한류학이 탄생했음을 알려준다.\n조영신 (SK브로드밴드 경영전략그룹장)\n담담하고도 대담하게 한류의 탄생과 성장을 그린 대서사극이다. 미래에서 본 과거의 역사물이자 갈 길을 보여주는 SF물이다.\n심재명 (명필름 대표)\nK콘텐츠로 불리게 된 한국 대중문화산업, 그 역동적인 30년의 시간을 생생하게 살려낸 성실한 기록이자 산업의 역사를 이해하기에 손색없는 전문 서적이다.\n펼치기'

In [3]:
# publisher_review = driver.find_element(By.XPATH,'//*[@id="scrollSpyProdInfo"]/div[9]/div[2]/div[1]/div/p').text
# publisher_review = driver.find_element(By.XPATH,'//*[@id="scrollSpyProdInfo"]/div[11]/div[2]/div[1]/div/p').text
publisher_review = driver.find_element(By.CLASS_NAME,'product_detail_area.book_publish_review').text
publisher_review


'출판사 서평\n“세상은 소비하는 사람이 아니라 뭔가를 만들어내는 사람이 바꾼다!”\n경제적 자유로 가는 확실하고 구체적인 실전 로드맵\n\n나름대로 열심히, 바쁘게 살아온 것 같은데 어느덧 직장경력 10여 년, 생각보다 손에 쥐고 있는 것이 별로 없다고 느끼는 사람들이 꽤 많다. 쌓이는 연차만큼 연봉도 올라가지만 갚아야 할 아파트 대출금은 쉽게 줄어들지 않고, 이제는 직급이 낮진 않지만 회사 밖에서 써먹을 만한 자기 브랜드도, 전문성도 가지고 있는 것 같지 않다. 자기계발에 소홀한 것도 아니고, 책도 꽤 읽고 공부도 한 것 같은데 왜 이렇게 회사를 벗어나서는 할 줄 아는 게 없다고 느껴지는 걸까?\n정년은커녕 5년 후도 불안한 지금, 3040에게 가장 필요한 능력은 무엇일까? 데이터를 이용한 부동산 투자로 유명한 재테크 전문가 렘군은 그의 첫 자기계발서 《아웃풋 법칙》을 통해 말한다. 이제 그만 좀 공부하고 제발 그 지식을 세상 밖으로 내놓으라고.\n\n지난 5년간 렘군은 ‘시작 컨설팅’이라는 이름으로 평범한 사람들이 자신의 강점을 찾고 제2의 인생 설계를 할 수 있도록 도왔다. 그는 이 과정을 통해 얻은 지식을 ‘아웃풋(output)’이라는 단어로 설명한다. 아웃풋은 단순히 어떤 생산물 또는 결과물만을 의미하지 않는다. 저자는 이를 타인을 위해, 세상과 연결되기 위해 제공하는 생산물로 정리한다.\n많은 사람이 ‘나는 부족해’ ‘아직 내 지식은 세상에 내놓을 것이 못 돼’ ‘좀 더 공부해야 해’ 하면서 더 훌륭해진 후에야 뭔가를 하려고 한다. 그러나 그 시기는 절대 오지 않는다고 저자는 잘라 말한다. 혼자서 최고의 완성도를 올리기 위해 노력하기보다는 끊임없이 세상에 무엇인가를 내놓으면서 ‘타인 혹은 세상이 원하는 것’이 무엇인지를 찾으라고 강조한다. 완벽하지 않더라도 자신의 아웃풋을 세상에 내놓는 바로 그 순간, ‘소비자의 영역’에서 벗어나 ‘생산자의 영역’으로 진입하게 되고, 그제야 비로소 삶에 변화가 일어난다는 사실을 알려준다.\n\n“언제까지 미련하게 

In [81]:
# ISBN = driver.find_element(By.XPATH,'//*[@id="scrollSpyProdInfo"]/div[10]/div[2]/table/tbody/tr[1]/td').text
# ISBN = driver.find_element(By.XPATH,'//*[@id="scrollSpyProdInfo"]/div[12]/div[2]/table/tbody/tr[1]/td').text
ISBN = driver.find_element(By.CLASS_NAME,'tbl_row').find_elements(By.TAG_NAME,'td')[0].text
ISBN

'9791167740977'

### flask로 띄워보기

In [163]:
from flask import Flask

In [47]:
detail_img = driver.find_element(By.CLASS_NAME,'product_detail_area.detail_img').get_attribute('outerHTML')
detail_img

'<div class="product_detail_area detail_img">\n    <div class="inner">\n        <img loading="lazy" src="https://contents.kyobobook.co.kr/sih/fit-in/814x0/dtl/illustrate/779/i9788947548779.jpg" alt="초거대 위협 상세 이미지">\n        \n    </div>\n</div>'

In [29]:
content = driver.find_element(By.XPATH,'//*[@id="scrollSpyProdInfo"]/div[11]').get_attribute('outerHTML')
content

'<div class="product_detail_area book_contents">\n    <div class="title_wrap title_size_md">\n        <h2 class="title_heading">목차</h2>\n    </div>\n    <div class="auto_overflow_wrap overflow">\n        <div class="auto_overflow_contents">\n            <div class="auto_overflow_inner">\n                <ul class="book_contents_list">\n                    <li class="book_contents_item">들어가는 글 <br><br>1부 거대 스태그플레이션과 부채 위기<br><br>1장 눈먼 시장이 불러들인 부채 위기 <br>벼랑 끝에 몰린 낙관주의자들 │ 반복되는 위기의 역사 │ 부채 팬데믹은 이미 시작되었다 <br><br>2장 민간 및 공공 부문 정책의 실패 <br>잘못된 정책이 낳은 세 가지 불일치 │ 부채 위기를 완전히 벗어날 방법은 없다 │ 부채의 빙산 아래 숨은 비용들 <br><br>3장 인구통계학적 시한폭탄 <br>세계의 고령화 그리고 바닥난 연금 │ 고령화 시대의 딜레마 │ 이민자 앞에 닫힌 문 <br><br>4장 저금리의 함정 그리고 호황과 불황의 주기 <br>호황과 불황의 주기는 게임이 아니다 │ 야성적 충동을 조장하는 느슨한 통화 정책 │ 금융 붕괴는 경제적 ‘실패’가 아닌 ‘인재’다 │ 좋은 디플레이션, 나쁜 디플레이션, 끔찍한 디플레이션 │ 쉬운 돈과 느슨한 정책의 함정에 빠진 중앙은행들<br><br>5장 거대 스태그플레이션의 도래 <br>1970년대 스태그플레이션의 악몽 │ 눈앞에 다가온 스태그플레이션의 징조들 │ 세계를 불황에 빠트릴 11가지 충격 <br><br>2부 금융, 무역, 지정학, 첨단기술, 환경의 위기<br><br>6장 통화 붕괴와 금융 불안

In [51]:
content = driver.find_element(By.CLASS_NAME,'product_detail_area.book_contents')
h2 = content.find_element(By.CLASS_NAME,'title_heading').get_attribute('outerHTML')
contents_list = content.find_element(By.CLASS_NAME,'auto_overflow_wrap.overflow').find_element(By.CLASS_NAME,'auto_overflow_contents').get_attribute('outerHTML')
h2,contents_list

('<h2 class="title_heading">목차</h2>',
 '<div class="auto_overflow_contents">\n            <div class="auto_overflow_inner">\n                <ul class="book_contents_list">\n                    <li class="book_contents_item">들어가는 글 <br><br>1부 거대 스태그플레이션과 부채 위기<br><br>1장 눈먼 시장이 불러들인 부채 위기 <br>벼랑 끝에 몰린 낙관주의자들 │ 반복되는 위기의 역사 │ 부채 팬데믹은 이미 시작되었다 <br><br>2장 민간 및 공공 부문 정책의 실패 <br>잘못된 정책이 낳은 세 가지 불일치 │ 부채 위기를 완전히 벗어날 방법은 없다 │ 부채의 빙산 아래 숨은 비용들 <br><br>3장 인구통계학적 시한폭탄 <br>세계의 고령화 그리고 바닥난 연금 │ 고령화 시대의 딜레마 │ 이민자 앞에 닫힌 문 <br><br>4장 저금리의 함정 그리고 호황과 불황의 주기 <br>호황과 불황의 주기는 게임이 아니다 │ 야성적 충동을 조장하는 느슨한 통화 정책 │ 금융 붕괴는 경제적 ‘실패’가 아닌 ‘인재’다 │ 좋은 디플레이션, 나쁜 디플레이션, 끔찍한 디플레이션 │ 쉬운 돈과 느슨한 정책의 함정에 빠진 중앙은행들<br><br>5장 거대 스태그플레이션의 도래 <br>1970년대 스태그플레이션의 악몽 │ 눈앞에 다가온 스태그플레이션의 징조들 │ 세계를 불황에 빠트릴 11가지 충격 <br><br>2부 금융, 무역, 지정학, 첨단기술, 환경의 위기<br><br>6장 통화 붕괴와 금융 불안 <br>달러의 무기화, 중국의 탈동조화 │ 암호화폐 그리고 탈중앙화 금융에 대한 그릇된 맹신 │ ‘가격’을 아는 것과 ‘가치’를 아는 것은 다르다 <br><br>7장 세계화의 종말 <br>일자리를 지키기 위한 탈세계화의 역효과 │ 세계 경제의 파이를 키운 자유무역 │ 사라지는 일자리에 분노하

In [26]:
pub_review = driver.find_element(By.CLASS_NAME,'product_detail_area.book_publish_review')
pub_review.find_element(By.CLASS_NAME,'').get_attribute('outerHTML')
pub_review

'<div class="product_detail_area book_publish_review">\n    <div class="title_wrap title_size_md">\n        <h2 class="title_heading">출판사 서평</h2>\n    </div>\n    <div class="auto_overflow_wrap type_text overflow">\n        <div class="auto_overflow_contents">\n            <div class="auto_overflow_inner">\n                <p class="info_text">· \'닥터 둠\' 누리엘 루비니 신작<br>· 김영익, 오건영 추천<br>· 나심 니콜라스 탈레브, 케네스 로고프, 애덤 그랜트 등 추천<br><br>2008년 금융 위기를 예견한 경제학자,<br>누리엘 루비니의 귀환!<br><br>· 출간 즉시 아마존 베스트셀러 <br>· 〈타임스〉 선정 올해의 비즈니스 도서 <br>· 〈파이낸셜 타임스〉 선정 올해의 경제학 도서 <br><br>누리엘 루비니의 이름 앞에는 수많은 수식어가 붙는다. 가장 맨 앞에 그리고 자주 붙는 것은 ‘2008년 금융 위기를 예견한 경제학자’라는 수식이다. 그는 2006년부터 줄기차게 세계 경제의 위기를 경고했고, 얼마 지나지 않아 미국에서 시작된 불황은 전 세계를 강타했다. 항상 비관적인 전망만 내놓는다는 뜻인 ‘닥터 둠’이라는 별명도 당시 IMF 회의 등에서 거품 붕괴를 주장했을 때 붙은 것이다. 여기에 각종 지면과 방송에 가장 자주 등장하는 경제학자라는 수식도 추가할 수 있겠다. 경제 현황을 발 빠르게 분석하는 논평가이자 경제연구소 수장 그리고 경제·금융·국제관계 관련 포럼의 단골 연사이기도 하다. 그리고 현재 뉴욕대학교 스턴 경영대학원 경제학과 명예교수로 재직 중이다. 뉴욕대학교 부임 전에는 예일대학교 경제학과 교수를 지냈다.<br> 이런 그가 2010년에 쓴 《위기 경제학》

In [57]:
def recombination(classname):
    main = driver.find_element(By.CLASS_NAME,classname)
    h2 = main.find_element(By.CLASS_NAME,'title_heading').get_attribute('outerHTML')
    contents = main.find_element(By.CLASS_NAME,'auto_overflow_wrap.overflow').find_element(By.CLASS_NAME,'auto_overflow_contents').get_attribute('outerHTML')
    return h2 + '\n' + contents

detail_img = driver.find_element(By.CLASS_NAME,'product_detail_area.detail_img').get_attribute('outerHTML')
contents_lst = recombination('product_detail_area.book_contents')
pub_review = recombination('product_detail_area.book_publish_review')
detail_page = [detail_img,contents_lst,pub_review]
detail_page

['<div class="product_detail_area detail_img">\n    <div class="inner">\n        <img loading="lazy" src="https://contents.kyobobook.co.kr/sih/fit-in/814x0/dtl/illustrate/779/i9788947548779.jpg" alt="초거대 위협 상세 이미지">\n        \n    </div>\n</div>',
 '<h2 class="title_heading">목차</h2>\n<div class="auto_overflow_contents">\n            <div class="auto_overflow_inner">\n                <ul class="book_contents_list">\n                    <li class="book_contents_item">들어가는 글 <br><br>1부 거대 스태그플레이션과 부채 위기<br><br>1장 눈먼 시장이 불러들인 부채 위기 <br>벼랑 끝에 몰린 낙관주의자들 │ 반복되는 위기의 역사 │ 부채 팬데믹은 이미 시작되었다 <br><br>2장 민간 및 공공 부문 정책의 실패 <br>잘못된 정책이 낳은 세 가지 불일치 │ 부채 위기를 완전히 벗어날 방법은 없다 │ 부채의 빙산 아래 숨은 비용들 <br><br>3장 인구통계학적 시한폭탄 <br>세계의 고령화 그리고 바닥난 연금 │ 고령화 시대의 딜레마 │ 이민자 앞에 닫힌 문 <br><br>4장 저금리의 함정 그리고 호황과 불황의 주기 <br>호황과 불황의 주기는 게임이 아니다 │ 야성적 충동을 조장하는 느슨한 통화 정책 │ 금융 붕괴는 경제적 ‘실패’가 아닌 ‘인재’다 │ 좋은 디플레이션, 나쁜 디플레이션, 끔찍한 디플레이션 │ 쉬운 돈과 느슨한 정책의 함정에 빠진 중앙은행들<br><br>5장 거대 스태그플레이션의 도래 <br>1970년대 스태그플레이션의 악몽 │ 눈앞에 다가온 스태그플레이션의 징조들 

In [65]:
style = '<style>ul{list-style:none;padding-left:0px;}</style>'
page = '<html>'+ style + '\n'.join(detail_page)+'</html>'

In [164]:
app = Flask(__name__)

@app.route("/")
def new():

    style = '<style>ul{list-style:none;padding-left:0px;}</style>'

    return '<html>'+ style + '\n'.join(detail_page)+'</html>'

@app.route("/exist")
def exist():
    text = '''
    <html><style>ul{list-style:none;padding-left:0px;}</style><div class="product_detail_area detail_img">
    <div class="inner">
        <img loading="lazy" src="https://contents.kyobobook.co.kr/sih/fit-in/814x0/dtl/illustrate/653/i9791157771653.jpg" alt="하룻밤에 읽는 그리스로마신화 상세 이미지">
        
    </div>
</div>
<h2 class="title_heading">목차</h2>
<div class="auto_overflow_inner">
                <ul class="book_contents_list">
                    <li class="book_contents_item">머리말 / 서양 문명의 원형이 살아 숨 쉬는 그리스·로마 신화<br><br>|제1부| 혼돈의 시대<br>카오스<br>대지의 형성<br>황금시대<br>은 시대<br>청동 시대와 철의 시대<br>기간토마키아<br>올림포스 회의<br>늑대로 변한 리카온 왕<br>인류를 멸망시키는 대홍수<br>데우칼리온과 피라<br>피톤을 무찌른 아폴론<br>아폴론과 다프네<br>제우스와 이오<br>시링크스와 판<br>공작 무늬가 된 아르고스<br>태양 마차를 모는 파에톤<br><br>|제2부| 신들의 시대<br>헬리아데스와 키크노스의 변신<br>칼리스토와 제우스<br>까마귀 깃털이 검어진 내력<br>말이 된 오키로에<br>돌로 변한 바토스 노인<br>헤르메스와 헤르세<br>제우스와 에우로페<br><br>|제3부| 광기의 시대<br>카드모스의 테베 건설<br>아르테미스와 악타이온<br>제우스와 세멜레<br>양성을 경험한 테이레시아스<br>나르키소스<br>신을 믿지 않은 펜테우스<br><br>|제4부| 탐욕의 시대<br>미니아스의 딸들<br>피라모스와 티스베<br>아프로디테와 아레스의 밀회<br>레우코토에와 클리티에<br>살마키스와 헤르마프로디토스<br>아타마스와 이노, 티시포네<br>페르세우스와 아틀라스<br>안드로메다와 바다 괴물 케토<br>메두사<br>피네우스의 반란<br><br> |제5부| 능욕의 시대<br>뮤즈를 괴롭혔던 피레네우스<br>페르세포네 납치<br>샘으로 변신한 아레투사<br><br>|제6부| 응징의 시대<br>아테나와 아라크네<br>니오베의 파멸<br>개구리가 된 리키아 농부들<br>마르시아스의 경연<br>펠롭스의 어깨<br>프로크네와 필로멜라의 복수<br>북풍의 신 보레아스의 혼인<br><br>|제7부| 영웅의 시대<br>아르고호 원정대<br>이아손과 메디아<br>아이손의 회춘<br>펠리아스의 희망과 죽음<br>메디아의 도주<br>아테네의 영웅 테세우스<br>미노스와 아이아코스<br>아이기나의 역병<br>케팔로스와 프로크리스<br><br>|제8부| 인간의 시대<br>니소스와 스킬라<br>미노타우로스<br>다이달로스와 이카로스<br>칼리돈의 멧돼지 사냥<br>필레몬과 바우키스<br>에리시크톤<br><br>|제9부| 영웅의 노래<br>헤라클레스의 선택<br>헤라클레스의 12과업<br>노예가 된 헤라클레스<br>신이 된 헤라클레스<br>오르페우스와 에우리디케<br>아폴론과 히아킨토스<br>피그말리온<br>미르라와 키니라스<br>아프로디테와 아도니스<br>아탈란테와 히포메네스<br>아도니스의 죽음<br>오르페우스의 죽음<br><br>|제10부| 변신의 시대<br>글라우코스와 스킬라<br>샘이 된 비블리스<br>이피스와 이안테<br>케익스와 알키오네<br>아키스와 갈라테이아<br>포플러가 된 드리오페<br>미다스의 손<br><br>[부록]<br>그리스·로마 신들의 대조표<br>태초 신들의 가계도<br>티탄 12신의 가계도<br>올림포스 12신의 가계도</li>
                </ul>
            </div>
<h2 class="title_heading">출판사 서평</h2>
<div class="auto_overflow_inner">
                <p class="info_text">서양 문명의 원형이 살아 숨 쉬는<br>그리스ㆍ로마 신화<br><br>신화에 접근하는 가장 간단한 방법은 그 주제를 보는 것이다. 가장 넓은 의미에서 신화는 신, 왕, 영웅 등에 대한 전통적인 이야기이다. 신화는 종종 세계의 창조와 연관되며, 때로는 미래의 멸망과 관련이 있다. 신화는 신들이 어떻게 사람을 창조했는지 알려주고, 다양한 신들과 인간 사이의 관계를 묘사하며, 인간이 살아가야 할 도덕률을 제공한다. 그리고 사회의 이상을 대표하는 영웅의 삶을 다룬다. 요컨대, 신화는 주로 인간과 초인적 존재의 중요한 측면들을 다룬다. <br>《그리스·로마 신화》는 《성경》을 제외하면 서양 문화에서 가장 풍부하고 비옥한 이야기 모음을 형성하는데, 그 기본 줄기는 그리스인들의 특성에 깊이 뿌리박고 있다. 그리스인들은 어차피 죽음이 불가피한 바에야 생명을 소중히 여기고 치열하게 살아야 한다고 믿었다. 예컨대 호메로스에게 죽음은 암울한 상태였지만, 삶 자체는 위험하고 긴장감 넘치면서도 영광스러운 것이었다. 평범한 사람들이든, 위대한 영웅들이든 간에 죽음을 피할 수 없다는 생각은 길가메시 서사시를 기록한 메소포타미아의 서기관들처럼 그리스인들을 슬프게 하지 않았다. 오히려 그리스인들은 열정적으로 반응했다. 그들은 죽음에 대한 유일한 해답은 장엄한 행위로 불멸의 전설을 새기는 것이라고 생각했다. <br>이리하여 그리스인들은 호메로스에서부터 알렉산드로스 대왕에 이르기까지 5세기 동안 놀라운 에너지로 명성을 추구했다. 그들은 강인하고, 불안하고, 야심적이고, 열심히 살고, 상상력이 풍부한 종족이었다. 또한 그리스인들의 남다른 명예욕은 그들로 하여금 죽음을 넘어서는 평판을 추구하게 만들었는데, 그들이 또한 열렬하고 복수심이 강했기 때문이다. 그리스인들의 이야기는 이러한 모든 특성을 풍부하게 보여준다.<br>올림포스의 신들은 그리스인들의 이러한 특성들을 충실하게 반영했으며, 다투기 좋아하고 용서하지 않는 신들로서 전쟁과 연회와 음행을 즐겼다. 그들은 항상 아름답고 강력한 몸을 드러낸 채 인간의 형태로 묘사되었다. 따라서 그들은 인간적으로 이해할 수 있을 뿐 아니라 눈으로 보기에도 매우 즐거웠다. 그리스인들은 힘, 아름다움, 지성을 크게 존경했다. 또한 그들에게 모든 것의 척도는 다름 아닌 인간이었는데, 이 때문에 그리스 신화는 인본주의적인 색채를 짙게 띠고 있다.<br>그리스 신화처럼 풍부한 영웅들을 배출한 신화는 거의 없다. 이것은 불멸의 명성을 추구하는 그리스인들의 성향에 비추어 보면 매우 자연스러운 결과였다. 영웅은 모험가이자 전사인 경향이 있다. 대담하고, 경험이 풍부하고, 치열하고, 강하고, 종종 영리하다. 그들의 업적은 평범한 인류의 업적보다 훨씬 뛰어나다. 그러나 그들은 또한 때때로 자신들을 망치는 심각한 결점을 안고 있다. 이를테면 과도한 자부심, 성급함, 잔인함과 같은 결함은 크나큰 야망에서 비롯되었다. 그리스 영웅들에게 야망은 강렬했고, 때로는 신과 같은 힘을 열망했다. 인간의 탁월한 모델로서 그들은 그리스 젊은이들이 본받을 표준을 제공했다.<br>비극적인 왕조의 전설은 이 같은 양면성을 보여준다. 이를테면 크레타, 미케네, 테베, 아테네의 왕족들은 그들 자신의 특정한 결점, 즉 권력에 대한 교만, 복수에 대한 무자비함, 어떤 목표를 추구하는 완고함, 성적 갈등으로 고통받고 있었다. 어떤 종족도 그리스인만큼 성격이 어떻게 운명을 결정 짓는지, 또는 우리의 성취가 어떻게 범죄와 같은 근원에서 비롯될 수 있는지 명확하게 이해하지 못했다. 결국 고대 그리스인들은 그들이 그토록 열렬히 추구했던 영구적인 명성을 얻었다. 그리고 그들의 신화는 2천 년이 훨씬 넘는 기간 동안 서양 예술과 문학의 주류를 형성해 왔다.<br>이 책은 오비디우스(BC 43~AD 17, 고대 로마의 시인)와 베르길리우스(BC 70~BC 19, 고대 로마의 시인)의 작품들을 텍스트로 하되, 신들의 명칭은 생소한 로마 신의 이름보다 더욱 친근하고 많이 쓰이고 있는 그리스 신들의 이름으로 표기하였다. 신화 스토리에 기본적인 연대기순 배치와 주제별 일람을 통해 그리스·로마 신들의 이야기가 보다 체계적이고 명확하게 정리될 수 있도록 하였고, 시각적이며 촉각적인 200여 점의 미술작품을 한 장씩 골라 보도록 신경을 썼다. 그러므로 독자들이 이 책을 읽어가는 동안 신화 이야기의 역동성과 함께 화려한 명화의 생동감을 함께 즐길 수 있기를 바란다.</p>
            </div></html>
    '''
    return text

In [165]:
host_addr = "0.0.0.0"
port_num = "8080"
app.run(host=host_addr,port=port_num)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://112.172.143.8:8080/ (Press CTRL+C to quit)
112.172.143.8 - - [29/May/2023 23:49:09] "GET / HTTP/1.1" 200 -
112.172.143.8 - - [29/May/2023 23:49:10] "GET /favicon.ico HTTP/1.1" 404 -
112.172.143.8 - - [29/May/2023 23:49:17] "GET /exist HTTP/1.1" 200 -


### DataFrame form

In [49]:
import pandas as pd
pd.read_excel('./form/신간주문_20230208-13 고도몰 등록폼(등록).xls')

,상품번호,상품명_기본,수수료,자체상품코드,상품명_메인,상품명_리스트,상품명_상세,상품명_제휴,상품명_영문몰,상품명_중문몰,...,교환 안내,상품 관리 메모,HS 코드,상품 개별 SEO 설정 사용여부,타이틀,메타태그 작성자,메타태그 설명,메타태그 키워드,페이스북 제품 피드 설정,페이스북 광고 제품 피드 이미지명
0,goods_no,goods_name,commission,goods_cd,name_main,name_list,name_detail,name_partner,global_data_2_goodsnm,global_data_3_goodsnm,...,detail_exchange,memo,hscode,seo_tag_fl,set_tag_title,set_tag_author,set_tag_description,set_tag_keyword,fb_vn,fb_image_name
1,10자리 숫자 (등록시에는 자동 생성 되므로 등록시에는 넣지 마세요),NaN,NaN,ISBN,NaN,NaN,NaN,NaN,입력시 별도의 해외상점 상품명이 등록됩니다.,입력시 별도의 해외상점 상품명이 등록됩니다.,...,NaN,NaN,해외상점 사용시 HS코드를 입력해 주세요.\n국가명^|^HS코드\n다중구분:Alt+...,"y:사용함, n:사용안함",NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,오둥이입니다만! : 나는 세상에서 가장 행복한 오리,NaN,9791191891256,NaN,NaN,NaN,NaN,NaN,NaN,...,005001,NaN,NaN,y,오둥이입니다만! : 나는 세상에서 가장 행복한 오리,NaN,NaN,NaN,NaN,NaN
3,NaN,심리학이 제갈량에게 말하다 1,NaN,9788972773702,NaN,NaN,NaN,NaN,NaN,NaN,...,005001,NaN,NaN,y,심리학이 제갈량에게 말하다 1,NaN,NaN,NaN,NaN,NaN
4,NaN,초거대 위협,NaN,9788947548779,NaN,NaN,NaN,NaN,NaN,NaN,...,005001,NaN,NaN,y,초거대 위협,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
539,NaN,20세기 현대 러시아 문학사,NaN,9791167971029,NaN,NaN,NaN,NaN,NaN,NaN,...,005001,NaN,NaN,y,20세기 현대 러시아 문학사,NaN,NaN,NaN,NaN,NaN
540,NaN,2024 최단 전공미술 서양미술사,NaN,9788962518634,NaN,NaN,NaN,NaN,NaN,NaN,...,005001,NaN,NaN,y,2024 최단 전공미술 서양미술사,NaN,NaN,NaN,NaN,NaN
541,NaN,좋아하는 일을 해도 괜찮을까?,NaN,9791139209082,NaN,NaN,NaN,NaN,NaN,NaN,...,005001,NaN,NaN,y,좋아하는 일을 해도 괜찮을까?,NaN,NaN,NaN,NaN,NaN
542,NaN,2024 하이패스 교직논술(기본편+기출·실전편)(전2권),NaN,9788962518641,NaN,NaN,NaN,NaN,NaN,NaN,...,005001,NaN,NaN,y,2024 하이패스 교직논술(기본편+기출·실전편)(전2권),NaN,NaN,NaN,NaN,NaN


In [160]:
driver = webdriver.Chrome()
driver.get('https://product.kyobobook.co.kr/detail/S000202040251')
book_name = driver.find_element(By.CLASS_NAME,'prod_title_box.auto_overflow_wrap').text
ISBN = driver.find_element(By.CLASS_NAME,'tbl_row_wrap').find_element(By.TAG_NAME,'td').text
book_name,ISBN

('왜 쓰는가', '9788954692489')

In [157]:
keyword_lst = []
# driver.find_element(By.CLASS_NAME,'prod_author_box.auto_overflow_wrap').find_elements(By.TAG_NAME,'a')[0].text
for i in driver.find_element(By.CLASS_NAME,'prod_author_box.auto_overflow_wrap').find_elements(By.TAG_NAME,'a'):
    print(i.text)
    keyword_lst.append(i.text)
keyword = ','.join(keyword_lst)
keyword

필립 로스
정영목


'필립 로스,정영목'

In [145]:
maker = driver.find_element(By.CLASS_NAME,'prod_info_text.publish_date').find_element(By.TAG_NAME,'a').text
maker

'서울미디어코믹스(서울문화사)'

In [131]:
date = driver.find_element(By.CLASS_NAME,'tbl_row_wrap').find_elements(By.TAG_NAME,'td')[1].get_attribute('innerHTML')
''.join(date.split()[:3])
# from datetime import datetime
# d = datetime.strptime(date,'%Y년 %m월 %d일')
# make_date = d.strftime('%Y%m%d')
# make_date

'2023년05월20일'

In [132]:
add_info = '저자^|^'+driver.find_element(By.CLASS_NAME,'prod_author_box.auto_overflow_wrap').text
add_info

'저자^|^이선종 엮음'

In [161]:
goods_price = driver.find_element(By.CLASS_NAME,'prod_price_box').text.split()[1][:-1].replace(',','')
fixed_price = driver.find_element(By.CLASS_NAME,'prod_price_box').text.split()[2][:-1].replace(',','')
goods_price,fixed_price

('25200', '28000')

In [162]:
main_img = driver.find_element(By.CLASS_NAME,'blur_img_box').find_element(By.TAG_NAME,'img').get_attribute('src')
main_img

'https://contents.kyobobook.co.kr/sih/fit-in/458x0/pdt/9788954692489.jpg'

In [135]:
main_img.split('/')[-1]

'9791157771653.jpg'

In [146]:
driver.find_element(By.CLASS_NAME,'product_detail_area.book_contents').find_element(By.CLASS_NAME,'auto_overflow_inner').get_attribute('outerHTML')

'<div class="auto_overflow_inner">\n                <ul class="book_contents_list">\n                    <li class="book_contents_item">제1화<br>제2화<br>제3화<br>제4화<br>제5화</li>\n                </ul>\n            </div>'

In [118]:
def recombination(classname):
    main = driver.find_element(By.CLASS_NAME,classname)
    h2 = main.find_element(By.CLASS_NAME,'title_heading').get_attribute('outerHTML')
    contents = main.find_element(By.CLASS_NAME,'auto_overflow_wrap.overflow').find_element(By.CLASS_NAME,'auto_overflow_inner').get_attribute('outerHTML')
    return h2 + '\n' + contents

detail_img = driver.find_element(By.CLASS_NAME,'product_detail_area.detail_img').get_attribute('outerHTML')
contents_lst = recombination('product_detail_area.book_contents')
try:
    pub_review = recombination('product_detail_area.book_publish_review')
except:
    pub_review = ''
detail_page = [detail_img,contents_lst,pub_review]
style = '<style>ul{list-style:none;padding-left:0px;}</style>'

goods_desc = '<html>'+ style + '\n'.join(detail_page)+'</html>'

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".product_detail_area.detail_img"}
  (Session info: chrome=113.0.5672.127)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00DD8893+48451]
	(No symbol) [0x00D6B8A1]
	(No symbol) [0x00C75058]
	(No symbol) [0x00CA0467]
	(No symbol) [0x00CA069B]
	(No symbol) [0x00CCDD92]
	(No symbol) [0x00CBA304]
	(No symbol) [0x00CCC482]
	(No symbol) [0x00CBA0B6]
	(No symbol) [0x00C97E08]
	(No symbol) [0x00C98F2D]
	GetHandleVerifier [0x01038E3A+2540266]
	GetHandleVerifier [0x01078959+2801161]
	GetHandleVerifier [0x0107295C+2776588]
	GetHandleVerifier [0x00E62280+612144]
	(No symbol) [0x00D74F6C]
	(No symbol) [0x00D711D8]
	(No symbol) [0x00D712BB]
	(No symbol) [0x00D64857]
	BaseThreadInitThunk [0x762000C9+25]
	RtlGetAppContainerNamedObjectPath [0x76ED7B4E+286]
	RtlGetAppContainerNamedObjectPath [0x76ED7B1E+238]


In [70]:
main = f'''{main_img}
{main_img}
{main_img}
'''
main

'https://contents.kyobobook.co.kr/sih/fit-in/458x0/pdt/9791192376004.jpg\nhttps://contents.kyobobook.co.kr/sih/fit-in/458x0/pdt/9791192376004.jpg\nhttps://contents.kyobobook.co.kr/sih/fit-in/458x0/pdt/9791192376004.jpg\n'

- column에 추가될 항목 :  
    book_name,ISBN,keyword,maker,make_date,add_info,goods_price,fixed_price,main_img,goods_desc

In [62]:
# ex_df = pd.read_excel('신간주문_고도몰_default.xlsx')
ex_df

,Unnamed: 0,상품번호,상품명_기본,수수료,자체상품코드,상품명_메인,상품명_리스트,상품명_상세,상품명_제휴,상품명_영문몰,...,교환 안내,상품 관리 메모,HS 코드,상품 개별 SEO 설정 사용여부,타이틀,메타태그 작성자,메타태그 설명,메타태그 키워드,페이스북 제품 피드 설정,페이스북 광고 제품 피드 이미지명
0,0,goods_no,goods_name,commission,goods_cd,name_main,name_list,name_detail,name_partner,global_data_2_goodsnm,...,detail_exchange,memo,hscode,seo_tag_fl,set_tag_title,set_tag_author,set_tag_description,set_tag_keyword,fb_vn,fb_image_name
1,1,10자리 숫자 (등록시에는 자동 생성 되므로 등록시에는 넣지 마세요),NaN,NaN,ISBN,NaN,NaN,NaN,NaN,입력시 별도의 해외상점 상품명이 등록됩니다.,...,NaN,NaN,해외상점 사용시 HS코드를 입력해 주세요.\n국가명^|^HS코드\n다중구분:Alt+...,"y:사용함, n:사용안함",NaN,NaN,NaN,NaN,NaN,NaN
2,2,NaN,하룻밤에 읽는 서양 철학,NaN,9791192376004,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,하룻밤에 읽는 서양 철학,NaN,NaN,NaN,NaN,NaN


In [71]:
img_store = 'https://jigoobooks.smilecast.co.kr/Book_cover_sqr/'
add_col =['상품명_기본','자체상품코드','검색,키워드','제조사','원산지','제조일','출시일','추가항목','과세/면세','판매가','정가','이미지 저장소','이미지명','PC쇼핑몰 상세 설명','모바일쇼핑몰 상세 설명','ISBN코드','타이틀']
ex_df.loc[2,add_col] = [book_name,ISBN,keyword,maker,'한국',make_date,make_date,add_info,'f',goods_price,fixed_price,img_store,main,goods_desc,goods_desc,ISBN,book_name]

In [72]:
e = ex_df[2:]
e

,Unnamed: 0,상품번호,상품명_기본,수수료,자체상품코드,상품명_메인,상품명_리스트,상품명_상세,상품명_제휴,상품명_영문몰,...,교환 안내,상품 관리 메모,HS 코드,상품 개별 SEO 설정 사용여부,타이틀,메타태그 작성자,메타태그 설명,메타태그 키워드,페이스북 제품 피드 설정,페이스북 광고 제품 피드 이미지명
2,2,NaN,하룻밤에 읽는 서양 철학,NaN,9791192376004,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,하룻밤에 읽는 서양 철학,NaN,NaN,NaN,NaN,NaN


In [84]:
from openpyxl import load_workbook
ori = '신간주문_고도몰_default.xlsx'
copy = '신간주문_고도몰_new2.xlsx'

w = load_workbook(ori)
w.save(copy)
w.close()

In [74]:
with pd.ExcelWriter('신간주문_고도몰_test.xlsx',mode='a',engine='openpyxl',if_sheet_exists='overlay') as w:
    e.to_excel(w,header=False,index=False,startrow=3)

고도몰 기준 : 카테고리 코드,원산지,추가항목(저자에서 중간. 포함되도 되는지),이미지명(이미지 비율조정때문에 사용?)  
- 이셀러스 :  
        원본번호 중간에 비는것  
        카테고리 번호  
        

### 신상도서 페이지 순회
    23년 5월 4주차 기준 url : https://product.kyobobook.co.kr/new/#?page=1&sort=new&year=2023&month=05&week=4&per=20&saleCmdtDvsnCode=KOR&gubun=newGubun&saleCmdtClstCode=
    1. 1페이지 각 도서별 url 수집
    2. 다음 페이지 넘어간 후 1. 기능을 수행 or url #?page={number}로 (className page_num의 text를 number 리스트화) 
        ---> 새창으로 열려서 worker가 작업하는게 아니라 nosuchelement 따라서 url정보만 리스트로 수집한 후 하나씩 돌리도록 설정해야할 것 같음.

In [4]:
driver.get('https://product.kyobobook.co.kr/new')
driver.current_url

'https://product.kyobobook.co.kr/new#?page=1&sort=new&year=2023&month=05&week=4&per=20&saleCmdtDvsnCode=KOR&gubun=newGubun&saleCmdtClstCode='

In [42]:
from urllib.parse import urlparse,parse_qsl,urlunparse,urlencode,quote
n = 2    
url = driver.current_url
q = urlparse(url,allow_fragments=False).query
unpack = dict(parse_qsl(q))
unpack['page'] = n
'https://product.kyobobook.co.kr/new#?'+urlencode(unpack)
# # urlunparse(unpack)

'https://product.kyobobook.co.kr/new#?page=2&sort=new&year=2023&month=05&week=4&per=20&saleCmdtDvsnCode=KOR&gubun=newGubun'

In [2]:
year = 2023
month = '05'
week = 4
page = 1
main_url = f'https://product.kyobobook.co.kr/new/#?page={page}&sort=new&year={year}&month={month}&week={week}&per=20&saleCmdtDvsnCode=KOR&gubun=newGubun&saleCmdtClstCode='
driver = webdriver.Chrome()
# driver.get('https://product.kyobobook.co.kr/new/#?page=1&sort=new&year=2023&month=05&week=4&per=20&saleCmdtDvsnCode=KOR&gubun=newGubun&saleCmdtClstCode=')
driver.get(main_url)

In [7]:
#page num list create
num = driver.find_element(By.CLASS_NAME,'page_num').text
splitnum = num.split()
num_str = list(str(int(splitnum[1])))
num_str.insert(0,splitnum[0]) #첫 페이지는 디폴트
num_str.append(splitnum[-1])
numbers = list(map(int,num_str))
numbers

[1, 2, 3, 4, 5, 6, 7]

In [8]:
url_list = []
for n in numbers:
    page = n
    driver.get(f'https://product.kyobobook.co.kr/new/#?page={page}&sort=new&year={year}&month={month}&week={week}&per=20&saleCmdtDvsnCode=KOR&gubun=newGubun&saleCmdtClstCode=')
    # time.sleep(3)
    WebDriverWait(driver,20).until(EC.presence_of_element_located((By.CLASS_NAME,'prod_list')))
    for item in driver.find_element(By.CLASS_NAME,'prod_list').find_elements(By.CLASS_NAME,'prod_item'):
        url = item.find_element(By.CLASS_NAME,'prod_area.horizontal').find_element(By.CLASS_NAME,'prod_thumb_box.size_lg').find_element(By.TAG_NAME,'a').get_attribute('href')
        url_list.append(url)

In [9]:
len(url_list)

136

In [10]:
url_list

['https://product.kyobobook.co.kr/detail/S000201866441',
 'https://product.kyobobook.co.kr/detail/S000202215993',
 'https://product.kyobobook.co.kr/detail/S000201980554',
 'https://product.kyobobook.co.kr/detail/S000202040251',
 'https://product.kyobobook.co.kr/detail/S000202267455',
 'https://product.kyobobook.co.kr/detail/S000202248749',
 'https://product.kyobobook.co.kr/detail/S000201891243',
 'https://product.kyobobook.co.kr/detail/S000201849929',
 'https://product.kyobobook.co.kr/detail/S000202183116',
 'https://product.kyobobook.co.kr/detail/S000201902542',
 'https://product.kyobobook.co.kr/detail/S000202006858',
 'https://product.kyobobook.co.kr/detail/S000202180398',
 'https://product.kyobobook.co.kr/detail/S000201992872',
 'https://product.kyobobook.co.kr/detail/S000202041226',
 'https://product.kyobobook.co.kr/detail/S000201940368',
 'https://product.kyobobook.co.kr/detail/S000202181040',
 'https://product.kyobobook.co.kr/detail/S000202205091',
 'https://product.kyobobook.co.